### Movies for Along Shelf Flux 

In [1]:
#KRM

import numpy as np

import matplotlib.pyplot as plt

import matplotlib.colors as mcolors

import matplotlib as mpl
#%matplotlib inline
from math import *

import scipy.io

import scipy as spy

from netCDF4 import Dataset

import pylab as pl

import os 

import sys

import seaborn as sns

In [2]:
lib_path = os.path.abspath('../../Building_canyon/BuildCanyon/PythonModulesMITgcm') # Add absolute path to my python scripts
sys.path.append(lib_path)

import ReadOutTools_MITgcm as rout 


In [3]:
def vTracAlong(Mask,V,zlim=30, ylim=230):
    '''MaskC : mask corresponding to '''
    mask_expand2 = np.expand_dims(Mask[:zlim,ylim,:],0)
    mask_expand2 = mask_expand2 + np.zeros(V[:,:zlim,ylim,:].shape)

    VTRACbox= np.ma.masked_array(V[:,:zlim,ylim,:],mask = mask_expand2)
    return(VTRACbox)



In [4]:
NoCCanyonGrid='/ocean/kramosmu/MITgcm/TracerExperiments/CNTDIFF/run07/gridGlob.nc'
NoCCanyonGridOut = Dataset(NoCCanyonGrid)

NoCCanyonState='/ocean/kramosmu/MITgcm/TracerExperiments/CNTDIFF/run07/stateGlob.nc'
NoCCanyonStateOut = Dataset(NoCCanyonState)

FluxTR01NoC = '/ocean/kramosmu/MITgcm/TracerExperiments/CNTDIFF/run07/FluxTR01Glob.nc'
NoCFluxOut = Dataset(FluxTR01NoC)

CanyonGrid='/ocean/kramosmu/MITgcm/TracerExperiments/CNTDIFF/run03/gridGlob.nc' # this has a canyon but calling it no canyon to use old code
CanyonGridOut = Dataset(CanyonGrid)

CanyonState='/ocean/kramosmu/MITgcm/TracerExperiments/CNTDIFF/run03/stateGlob.nc'
CanyonStateOut = Dataset(CanyonState)

FluxTR01 = '/ocean/kramosmu/MITgcm/TracerExperiments/CNTDIFF/run03/FluxTR01Glob.nc'
FluxOut = Dataset(FluxTR01)



In [5]:
#for dimobj in CanyonStateOut.variables.values():
#    print dimobj

In [6]:
# General input

nx = 360
ny = 360
nz = 90
nt = 19 # t dimension size 

z = CanyonStateOut.variables['Z']
Time = CanyonStateOut.variables['T']

xc = rout.getField(CanyonGrid, 'XC') # x coords tracer cells
yc = rout.getField(CanyonGrid, 'YC') # y coords tracer cells

bathy = rout.getField(CanyonGrid, 'Depth')

hFacC = rout.getField(CanyonGrid, 'HFacC')
MaskC = rout.getMask(CanyonGrid, 'HFacC')
MaskCNoC = rout.getMask(NoCCanyonGrid, 'HFacC')



In [7]:
VTR = rout.getField(FluxTR01,'VTRAC01') # 
UTR = rout.getField(FluxTR01,'UTRAC01') # 
UTRAC,VTRAC = rout.unstagger(UTR, VTR)

VTR = rout.getField(FluxTR01NoC,'VTRAC01') # 
UTR = rout.getField(FluxTR01NoC,'UTRAC01') # 
UTRACNoC,VTRACNoC = rout.unstagger(UTR, VTR)

#WTRAC = rout.getField(FluxTR01,'ADVrTr01') # 
#WTRACNoC = rout.getField(FluxTR01NoC,'ADVrTr01') # 


In [8]:
CSbase = vTracAlong(MaskC,VTRAC,zlim=30,ylim=230)*1000.0
CSbaseNoC = vTracAlong(MaskCNoC,VTRACNoC,zlim=30,ylim=230)*1000.0
Anom = (CSbase-CSbaseNoC)

In [9]:
minT = CSbase.min()
maxT = CSbase.max()

minTNoC = CSbaseNoC.min()
maxTNoC = CSbaseNoC.max()

minTAnom = Anom.min()
maxTAnom = Anom.max()

print(minT, maxT)
print(minTNoC, maxTNoC)
print(minTAnom,maxTAnom)


(-711.20728, 757.87689)
(-37.202591, 273.61804)
(-934.50385, 591.75635)


In [10]:
import matplotlib.animation as animation

In [11]:
sns.set()
sns.set_style('white')
sns.set_context("talk")


#divmap = sns.diverging_palette(255, 100, l=60, n=7, center="dark", as_cmap=True)

In [26]:
def Plot1(t,ax1):
    ax1.clear()
    csU = np.linspace(-maxT,maxT,num=31)
    Base = ax1.contourf(xc[230,:], z[:30],CSbase[t,:,:],csU,cmap='RdYlGn')
    
    if t == 1:    
        cbar=plt.colorbar(Base,ax=ax1,ticks=[np.arange(-maxT,maxT,250)])
        cbar.set_label('$ mol \cdot m /l \cdot s$')
        
    #CS = ax1.contour(yc[100:-1,200],z[:58],Uplot[:58,100:]/Umax,csU2,colors='k',linewidths=[0.75] )
    ax1.set_axis_bgcolor((205/255.0, 201/255.0, 201/255.0))
    ax1.set_xlabel('Along-shore distance [km]')
    ax1.set_xticks([10000,20000,30000,40000,50000,60000,70000,80000,90000,100000,110000,120000])
    ax1.set_xticklabels(['10','20','30','40', '50', '60', '70', '80','90','100','110','120'])
    ax1.set_ylabel('Depth [m]')
    ax1.set_title('Base case - Cross-shore transport at day %0.1f' %(t/2.0+0.5))

def Plot2(t,ax2):
    ax2.clear()
    csU = np.linspace(-maxTNoC,maxTNoC,num=31)
    Base = ax2.contourf(xc[230,:], z[:30],CSbaseNoC[t,:,:],csU,cmap='RdYlGn')
    
    if t == 1:    
        cbar=plt.colorbar(Base,ax=ax2,ticks=[np.arange(-maxTNoC,maxTNoC,100)])
        cbar.set_label('$mol \cdot m /l \cdot s$')
        
    #CS = ax1.contour(yc[100:-1,200],z[:58],Uplot[:58,100:]/Umax,csU2,colors='k',linewidths=[0.75] )
    ax2.set_axis_bgcolor((205/255.0, 201/255.0, 201/255.0))
    ax2.set_xlabel('Along-shore distance [km]')
    ax2.set_xticks([10000,20000,30000,40000,50000,60000,70000,80000,90000,100000,110000,120000])
    ax2.set_xticklabels(['10','20','30','40', '50', '60', '70', '80','90','100','110','120'])
    ax2.set_ylabel('Depth [m]')
    ax2.set_title('No canyon case')

def Plot3(t,ax3):
    ax3.clear()
    csU = np.linspace(minTAnom,-minTAnom,num=31)
    Base = ax3.contourf(xc[230,:], z[:30],Anom[t,:,:],csU,cmap='RdYlBu')
    
    if t == 1:    
        cbar=plt.colorbar(Base,ax=ax3,ticks=[np.arange(minTAnom,-minTAnom,250)])
        cbar.set_label('$mol \cdot m /l \cdot s$')
        
    #CS = ax3.contour(yc[100:-1,200],z[:58],Uplot[:58,100:]/Umax,csU2,colors='k',linewidths=[0.75] )
    ax3.set_axis_bgcolor((205/255.0, 201/255.0, 201/255.0))
    ax3.set_xlabel('Along-shore distance [km]')
    ax3.set_xticks([10000,20000,30000,40000,50000,60000,70000,80000,90000,100000,110000,120000])
    ax3.set_xticklabels(['10','20','30','40', '50', '60', '70', '80','90','100','110','120'])
    ax3.set_ylabel('Depth [m]')
    ax3.set_title('Difference')




In [28]:
## Animation
#N=5
xslice=180
yslice=235
zslice= 29 # shelf break index
zslice2= 23 

#Empty figures
fig,((ax1),(ax2),(ax3))  = plt.subplots(3, 1)

#Initial image
def init():
    Plot1(0,ax1)
    Plot2(0,ax2)
    Plot3(0,ax3)
    
    plt.tight_layout()
    #return[ax1,ax2,ax3,ax4,ax5,ax6,ax7,ax8,ax9]
    

def animate(tt):
    Plot1(tt,ax1)
    Plot2(tt,ax2)
    Plot3(tt,ax3)
    plt.tight_layout()
#The animation function (max frames=47)   


anim = animation.FuncAnimation(fig, animate, init_func=init,frames=18, interval = 200,blit=False, repeat=False)

##A line that makes it all work
mywriter = animation.FFMpegWriter()

##Save in current folder
anim.save('TransportAlongTr01_Base_and_NoC.mp4',writer=mywriter,fps=0.2, dpi = 200, bitrate = 1000000, codec = "libx264")

plt.show()

